<center> <img src = "img\logo.png" alt="drawing" style="width:400px;">

<center>  

<span style="background-size: 600px;background:White;color:REd;font-size: 60px;font-family: Comic Sans MS">Кредитный скоринг Альфа банка</span>

# <span style="color:DeepSkyBlue">Задача</span>

**Задача**

Кредитный скоринг – важнейшая банковская задача. Стандартным подходом к ее решению   
является построение классических моделей машинного обучения, таких как логистическая   
регрессия и градиентный бустинг, на табличных данных, в том числе используя агрегации  
от каких-нибудь последовательных данных, например, транзакционных историй клиентов.   
Альтернативный подход заключается в использовании последовательных данных “как есть”,   
подавая их на вход рекуррентной нейронной сети.

В этом соревновании участникам предлагается решить задачу кредитного скоринга клиентов   
Альфа-Банка, используя только данные кредитных историй. [Источник](https://www.kaggle.com/competitions/alfa-bank-pd-credit-history)

**Данные**

Датасет соревнования устроен таким образом, что кредиты для тренировочной выборки взяты   
за период в М месяцев, а кредиты для тестовой выборки взяты за последующие K месяцев.

Каждая запись кредитной истории содержит самую разнообразную информацию о прошлом кредите   
клиента, например, сумму, отношение клиента к кредиту, дату открытия и закрытия, информацию   
о просрочках по платежам и др. Все публикуемые данные тщательно анонимизированы.

Целевая переменная – бинарная величина, принимающая значения 0 и 1, где 1 соответствует   
дефолту клиента по кредиту.


**Проверка решений**

Метрика соревнования – ROC AUC. Подробнее про метрику можно почитать, например, [здесь](https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/).

# <span style="color:DeepSkyBlue">Используемые библиотеки</span>

In [1]:
import os

# работа с регулярными выражениями
import re

# библиотеки для работы с табличными данными
import pandas as pd
import fastparquet as fp

# генерация случайных чисел
import random
from random import randint
from sklearn.utils import shuffle

# библиотеки для построения графики
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

# библиотеки для математических преобразований с массивами данных
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# библиотеки для работы с функциями(частичная передача аргументов в функцию)
from functools import partial

# библиотеки для работы со статистическими характеристиками
from scipy import stats
import statistics
from collections import Counter

# библиотеки для работы с pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

# Импортируем DBSCAN-кластеризацию
from sklearn.cluster import DBSCAN

# вставить картинку в Jupiter Notebook
from IPython.display import Image

# линейные модели машинного обучения
from sklearn import linear_model

# ансамбли моделей машинного обучения
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# поиск гиперпараметров модели
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
import optuna

 # метрики
from sklearn import metrics

# библиотека для стандартизации данных
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# сохранить полученные модели
import joblib
from joblib import dump, load

# сборщик мусора
import gc

# для ограничения времени выполнения функции
import signal
import func_timeout

# для отслеживания времени выполнения функции
import time

# очистить output
from IPython.display import clear_output

# <span style="color:DeepSkyBlue">Разведывательный анализ данных (EDA-Exploratory Data Analysis)</span>

## <span style="color:DodgerBlue">Исходные данные</span>

In [2]:
# Подгрузим данные о кредитной истории клиентов
td0 = pd.read_parquet('data_source/train_data/train_data_0.pq')
td1 = pd.read_parquet('data_source/train_data/train_data_1.pq')
td2 = pd.read_parquet('data_source/train_data/train_data_2.pq')
td3 = pd.read_parquet('data_source/train_data/train_data_3.pq')
td4 = pd.read_parquet('data_source/train_data/train_data_4.pq')
td5 = pd.read_parquet('data_source/train_data/train_data_5.pq')
td6 = pd.read_parquet('data_source/train_data/train_data_6.pq')
td7 = pd.read_parquet('data_source/train_data/train_data_7.pq')
td8 = pd.read_parquet('data_source/train_data/train_data_8.pq')
td9 = pd.read_parquet('data_source/train_data/train_data_9.pq')
td10 = pd.read_parquet('data_source/train_data/train_data_10.pq')
td11 = pd.read_parquet('data_source/train_data/train_data_11.pq')
# Обьединим данные в один датасет
train_data = pd.concat([td0,td1,td2,td3,td4,td5,td6,td7,td8,td9,td10,td11],ignore_index=True)

# Подгрузим данные о дефлоте клиента
target = pd.read_csv('data_source/train_data/train_target.csv')

In [3]:
# очистим память от ненужных данных
del td0,td1,td2,td3,td4,td5,td6,td7,td8,td9,td10,td11
gc.collect()

0

In [4]:
# посмотрим совпадают ли размерности датасетов
print(train_data.shape)
print(target.shape)

(26162717, 61)
(3000000, 2)


In [5]:
# Посмотрм на данные о кредитной истории клиентов
train_data.head(11)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,...,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,...,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,...,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,...,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,...,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,...,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,...,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,...,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,...,3,3,3,4,1,2,4,1,0,0


In [6]:
print(train_data.columns)

Index(['id', 'rn', 'pre_since_opened', 'pre_since_confirmed', 'pre_pterm',
       'pre_fterm', 'pre_till_pclose', 'pre_till_fclose',
       'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
       'pre_loans_outstanding', 'pre_loans_total_overdue',
       'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5',
       'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90',
       'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060',
       'is_zero_loans6090', 'is_zero_loans90', 'pre_util', 'pre_over2limit',
       'pre_maxover2limit', 'is_zero_util', 'is_zero_over2limit',
       'is_zero_maxover2limit', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2',
       'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7',
       'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12',
       'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16',
       'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20',
       'enc_paym_21', 

id	-	Идентификатор заявки(id -клиента).  
rn	-	Порядковый номер кредитного продукта в кредитной истории. Большему номеру соответствует продукт с более поздней датой открытия.	  

**date features:**  
1. pre_since_opened	-	Дней с даты открытия кредита до даты сбора данных (бинаризовано**)	  
2. pre_since_confirmed	-	Дней с даты подтверждения информации по кредиту до даты сбора данных (бинаризовано**)	  
3. pre_pterm	-	Плановое количество дней с даты открытия кредита до даты закрытия (бинаризовано**)	  
4. pre_fterm	-	Фактическое количество дней с даты открытия кредита до даты закрытия (бинаризовано**)	  
5. pre_till_pclose	-	Плановое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано**)	  
6. pre_till_fclose	-	Фактическое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано**)
7. pclose_flag	-	Флаг: плановое количество дней с даты открытия кредита до даты закрытия не определено 	  
8. fclose_flag	-	Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено

**late payments features:**  
1. pre_loans5	-	Число просрочек до 5 дней (бинаризовано**)	  
2. pre_loans530	-	Число просрочек от 5 до 30 дней (бинаризовано**)	  
3. pre_loans3060	-	Число просрочек от 30 до 60 дней (бинаризовано**)	  
4. pre_loans6090	-	Число просрочек от 60 до 90 дней (бинаризовано**)	  
5. pre_loans90	-	Число просрочек более, чем на 90 дней (бинаризовано**)	  
6. is_zero_loans5	-	Флаг: нет просрочек до 5 дней	  
7. is_zero_loans530	-	Флаг: нет просрочек от 5 до 30 дней	  
8. is_zero_loans3060	-	Флаг: нет просрочек от 30 до 60 дней	  
9. is_zero_loans6090	-	Флаг: нет просрочек от 60 до 90 дней	  
10. is_zero_loans90	-	Флаг: нет просрочек более, чем на 90 дней	  
11. pre_loans_total_overdue	-	Текущая просроченная задолженность (бинаризовано**)	  
12. pre_loans_max_overdue_sum	-	Максимальная просроченная задолженность (бинаризовано**)	


**credit features:** 
1. pre_loans_credit_limit	-	Кредитный лимит (бинаризовано**)	  
2. pre_loans_next_pay_summ	-	Сумма следующего платежа по кредиту (бинаризовано**)	  
3. pre_loans_outstanding	-	Оставшаяся невыплаченная сумма кредита (бинаризовано**)	  
4. pre_loans_credit_cost_rate	-	Полная стоимость кредита (бинаризовано**)	  
  

**relative features:**  
1. pre_util	-	Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту (бинаризовано**)	  
2. pre_over2limit	-	Отношение текущей просроченной задолженности к кредитному лимиту (бинаризовано**)	  
3. pre_maxover2limit	-	Отношенение максимальной просроченной задолженности к кредитному лимиту (бинаризовано**)	  
4. is_zero_util	-	Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равняется 0	  
5. is_zero_over2limit	-	Флаг: отношение текущей просроченной задолженности к кредитному лимиту равняется 0	  
6. is_zero_maxover2limit	-	Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равняется 0	  

**payments features:**   
1. enc_paym_{0..N}	-	Статусы ежемесячных платежей за последние N месяцев (закодировано***)	  

**service features:**  
1. enc_loans_account_holder_type	-	Тип отношения к кредиту (закодировано***)	  
2. enc_loans_credit_status	-	Статус кредита (закодировано***)	  
3. enc_loans_account_cur	-	Валюта кредита (закодировано***)	  
4. enc_loans_credit_type	-	Тип кредита (закодировано***)	  

  ** область значений поля разбивается на N непересекающихся промежутков, каждому промежутку случайным образом ставится в соответствие уникальный номер от 0 до N-1, значение поля заменяется номером промежутка, которому оно принадлежит  
  *** каждому уникальному значению поля случайным образом ставится в соответствие уникальный номер от 0 до K, значение поля заменяется номером этого значения		

In [7]:
# посмотрим на данные о дефолте клиента
target

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2999995,2999995,0
2999996,2999996,0
2999997,2999997,0
2999998,2999998,0


id	-	Идентификатор заявки(id -клиента).   
flag – Целевая переменная, 1 – факт ухода в дефолт.

In [8]:
# проверим целевую переменную на сбалансированность
target['flag'].value_counts(normalize=True)

flag
0    0.964519
1    0.035481
Name: proportion, dtype: float64

In [9]:
# тоже в абсолютных значениях
target['flag'].value_counts()

flag
0    2893558
1     106442
Name: count, dtype: int64

## <span style="color:DodgerBlue">Анализ данных</span>

Обратим внимание, что данные представлены в виде кредитной истории клиента.  
У каждого клиента своя "продолжительность" кредитной истории.  
Посмотрим "каких" историй больше для каждого типа клиентов (дефолт/не дефолт)  

In [ ]:
# Создадим рабочую плоскость на котором можно разместить две диаграммы одну под другой с общей осью абсцисс
fig_1 = make_subplots(
    rows=2, # количество диаграмм по вертикали
    cols=1, # количество диаграмм по горизонтали
    shared_xaxes=True, # использовать одну ось х для двух диаграмм
    vertical_spacing = 0.05, # уменьшим расстояние между диаграммами
    x_title= 'Количество клиетских операций',
)
# Настроем оформление рабочей поверхности
fig_1.update_layout(
    title ={
        'text':'Распределение количества клиентских операций (клиент без дефолта)', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1400, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )

# Добавим на рабочую плоскость коробчатую диаграмму
fig_1.add_box(
    x=train_data.groupby('id')['rn'].count().iloc[target[target['flag']==0].index], # Выбираем данные по оси х
    y0=0, # Устанавливаем значение в нуле по оси y
    row=1, col=1, # Выбираем в какой координатной плоскости будет строиться диаграмма
    showlegend=False, # убираем легенду (обе диаграммы представляют распределение одной величины)
    boxmean=True, # добавим на коробчатую диаграмму отметку среднего значения заработной платы
    name= '', # при наведение на значение не будет отображаться имя диаграммы
    hoverinfo='x' # при наведение отражается только значение заработной платы
) 

# Настроим отображение оси y для коробчатой диаграммы
fig_1.update_yaxes(
    title={'text':'Коробчатая диаграмма', # добавим название оси "y" для коробчатой диаграммы
            'font':{'size':20,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
            },
        ticklabelposition = 'inside top', # "спрячем" значения по оси "y"
    row=1,col=1) 

# Добавим на рабочую плоскость гистограмму c графиком оценки плотности ядра KDE
# Воспользуемся библиотекой plotly.figure_factory
# Формируем графику
dist_fig = ff.create_distplot(hist_data=[train_data.groupby('id')['rn'].count().iloc[target[target['flag']==0].index]], 
                              group_labels=[''],
                              bin_size=1,
                              show_rug=False,
                              show_hist=True,
                              histnorm='probability',
                              colors=['red'],
                              )
# Настроим отображение графики
dist_fig.update_traces(hovertemplate='Количество: %{x}<br>Плотность вероятности: %{y}', # добавить надписи при наведение
                       showlegend=False # уберем значок из легенды
                       )
# я не нашел как другим способом объединить subplots и figure_factory
# на сколько я разобрался, суть метода построения заключается в извлечение данных из figure_factory и добавление фрагментов в subplots
for trace in dist_fig.select_traces():
    fig_1.add_trace(trace, row=2, col=1)

# # Настроим отображение оси y для гистограммы
fig_1.update_yaxes(
     title ={
        'text':'Гистограмма распределения<br>(KDE-плотность вероятности)', # добавим название оси "y"
        'font':{'size':20,'family':"Times New Roman"}, # размер и стиль написания имени
        },
    ticklabelposition = 'inside top', # "спрячем" значения по оси "y"
    row=2,col=1) 

fig_1.show()

<center> <img src = "img\withoutdefolt.png" alt="drawing" style="width:1300px;">

<span style="color:Blue"> 

Вывод:  

Распределение количества клиентских операций отлично от нормального.  
Из коробчатой диаграммы мы наблюдаем наличие выбросов - клиенты с количеством опреций больше 24.  
Так же коробчатая диаграмма показывает, что среднее значение больше медианы распределения. В этом случае, график    
распределения "скошен" вправо относительно нормального, что и наблюдается на графике распределения плотности вероятности.  
В рамках разброса количество клиетских операций принимает значения от 1 до 24, при этом основная часть приходится на   
знаечния от 4 до 12. 
Медиальное значение равно 7, а среднее значение приближается к 9.


In [ ]:
# Создадим рабочую плоскость на котором можно разместить две диаграммы одну под другой с общей осью абсцисс
fig_1 = make_subplots(
    rows=2, # количество диаграмм по вертикали
    cols=1, # количество диаграмм по горизонтали
    shared_xaxes=True, # использовать одну ось х для двух диаграмм
    vertical_spacing = 0.05, # уменьшим расстояние между диаграммами
    x_title= 'Количество клиетских операций',
)
# Настроем оформление рабочей поверхности
fig_1.update_layout(
    title ={
        'text':'Распределение количества клиентских операций (клиент с дефолтом)', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1400, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )

# Добавим на рабочую плоскость коробчатую диаграмму
fig_1.add_box(
    x=train_data.groupby('id')['rn'].count().iloc[target[target['flag']==1].index], # Выбираем данные по оси х
    y0=0, # Устанавливаем значение в нуле по оси y
    row=1, col=1, # Выбираем в какой координатной плоскости будет строиться диаграмма
    showlegend=False, # убираем легенду (обе диаграммы представляют распределение одной величины)
    boxmean=True, # добавим на коробчатую диаграмму отметку среднего значения заработной платы
    name= '', # при наведение на значение не будет отображаться имя диаграммы
    hoverinfo='x' # при наведение отражается только значение заработной платы
) 

# Настроим отображение оси y для коробчатой диаграммы
fig_1.update_yaxes(
    title={'text':'Коробчатая диаграмма', # добавим название оси "y" для коробчатой диаграммы
            'font':{'size':20,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
            },
        ticklabelposition = 'inside top', # "спрячем" значения по оси "y"
    row=1,col=1) 

# Добавим на рабочую плоскость гистограмму c графиком оценки плотности ядра KDE
# Воспользуемся библиотекой plotly.figure_factory
# Формируем графику
dist_fig = ff.create_distplot(hist_data=[train_data.groupby('id')['rn'].count().iloc[target[target['flag']==0].index]], 
                              group_labels=[''],
                              bin_size=1,
                              show_rug=False,
                              show_hist=True,
                              histnorm='probability',
                              colors=['red'],
                              )
# Настроим отображение графики
dist_fig.update_traces(hovertemplate='Количество: %{x}<br>Плотность вероятности: %{y}', # добавить надписи при наведение
                       showlegend=False # уберем значок из легенды
                       )
# я не нашел как другим способом объединить subplots и figure_factory
# на сколько я разобрался, суть метода построения заключается в извлечение данных из figure_factory и добавление фрагментов в subplots
for trace in dist_fig.select_traces():
    fig_1.add_trace(trace, row=2, col=1)

# # Настроим отображение оси y для гистограммы
fig_1.update_yaxes(
     title ={
        'text':'Гистограмма распределения<br>(KDE-плотность вероятности)', # добавим название оси "y"
        'font':{'size':20,'family':"Times New Roman"}, # размер и стиль написания имени
        },
    ticklabelposition = 'inside top', # "спрячем" значения по оси "y"
    row=2,col=1) 

fig_1.show()

<center> <img src = "img\withdefolt.png" alt="drawing" style="width:1300px;">

<span style="color:Blue"> 

Вывод для клиентов с дефолтом:  

Распределение количества клиентских операций отлично от нормального.  
Из коробчатой диаграммы мы наблюдаем наличие выбросов - клиенты с количеством опреций больше 25.  
Так же коробчатая диаграмма показывает, что среднее значение больше медианы распределения. В этом случае, график    
распределения "скошен" вправо относительно нормального, что и наблюдается на графике распределения плотности вероятности.  
В рамках разброса количество клиетских операций принимает значения от 1 до 25, при этом основная часть приходится на   
знаечния от 3 до 12. 
Медиальное значение равно 7, а среднее значение приближается к 8.


## <span style="color:DodgerBlue">Анализ признаков</span>

Выводы по внешнему предварительному анализу данных:

Все признаки в датасете либо бинаризированы, либо закодированы - соответственно являются категориальными.  
Данных в датасете содержащием кредитные истории клиентов больше, чем в датасете содержащем информацию о дефолте клиента.  
Это связано с тем, что на каждого клиента существует кредитная история из нескольких кредитов.  
Для того, чтобы подать эти данные в модель необходимо агрегировать информацию по каждой клиенту.   
В качестве агрегирующих функций примем статистические характеристики случайных величин.   
Проаналировав поле признаков - его можно разделить на 6 групп.  
Помимо всего небходимо учитывать, что данные кредитных историй имееют хронологическую последовательность.  
Необходимо, не потерять информацию об этом при агригации данных.  
Так же необходимо не потерять информацию, о количестве  кредитов в истории.  
В связи с выше изложенным, примем следующую концепцию решения задачи:

Подготовка данных для обучения модели:
1. Выделетить часть данных для проверки модели (80/20 %).
2. Разбить признаки на 6 групп.
3. Внутри каждой группы в границах одного клиента выполниить преобразование признаков:  
    3.1. Заменить значения признаков на их статистичекие характеристики без учета хронологии кредитсной истории;  
    3.2. Добавить признак количества кредитных историй.  
    3.3. Создать признаки учитывающие историю клинетских операций.

## <span style="color:DodgerBlue">Постановка задачи в рамках EDA</span>

Данные о клиентах представлены в виде заявок, номера которых соотностятся с датами подачи заявки.  
Большему номеру соответствует более поздняя дата заявки.

Разделим процесс преобразования признаков на три части: 

1. Разделим признаки на под пространства.

2. Преобразуем кредитные операции клиента в его признаки, сохранив последовательность операций.  
Такие методы кодирования как Ordinal Encoding, OneHot Encoding не подойдут, так как уничтожится   
информация о последовательности операций.

3. Преобразование признаков как дискренного ряда.  
Для каждого признака в "границах" одного "id" рассчитать статистические показатели ряда, такие как:  
   - среднее значение ряда (математическое ожидание);  
   - средне гармоническое значение ряда;   
   - стандартное отклонение;  
   - минимальное значение;  
   - 25% квантиль;  
   - 50% квантиль;  
   - 75% квантиль;  
   - максимальное значение;  
   - мода ряда (среднее значение мод);  
   - частота появления моды (средненего значения мод);
   - и т.д. 


## <span style="color:DodgerBlue">Разработка инструментов преобразования данных</span>

### <span style="color:RoyalBlue">функция torow_transformer</span>

In [10]:
# функция torow_transformer

# Назначение: Преобразование признака столбца в признаки строки
#               с сохранением обратной последовательности в признаке.
#               (извлечение последних операций клиента)

# Внешние переменные функции: DataFrame, n_last
#   DataFrame - исоходый DataFrame
#   n_last - необходимое счисло послдених операций клиента
#   Структура DataFrame:
#       1. id
#       2. feature1
#       3. feature2
#       4. feature3
#       ...

# Результат работы функции: New DataFrame
#   Признаки New DataFrame:
#       1. id
#       2. feature1.1
#       3. feature1.2
#       4. feature1.3
#       ...
#       5. feature1.N
#       6. feature2.1
#       7. feature2.2
#       ...
#       8. feature2.N
#       ...
#   где featureX.1 - соотвествует последней операции клиента,
#       featureX.2 - соотвествует предпоследней операции клиента,
#       .....

# 2. dict_features - словарь (карта) признаков,
#    в котором отображается сокращения признаков и их расщифровка

# Алгоритм работы функции:
# 1. извлекаем признаки из данных DataFrame
# 3. формируем карту признаков dict_features:
#    3.1. каждый признак кодируется следующим образом: 'fn',
#         где n - порядковый номер признака
#    3.2. полные имена признаков задаются следующим образом: 'feature_N',
#         где N - порядок клиентской операции (большему N соотвествует, более ранняя операция)

# 4. Преобразуем данные к массиву
# 5. Группируем массив для каждого клиента
# 6. К групированному массиву прменяем следующие преобразования:
#    6.1. Обращаем порядок клиенских операций
#    6.2. выбираем посление n_last операций
#    6.3. Если число клиенски операций меньше чем n_last, дополняем их нулями
# 7. Преобразуем полученные данные к DataFrame

# Описание локальных переменных функции:
# 1. pd_data - исходный DataFrame
# 2. n_last - необходимое число последних операций клиента
# 3. list_id - список для хранения id клиентов внутри функции
# 4. list_features - список для хранения признаков исхдного dataframe внутри функции
# 5. dict_features - локальная карта признаков
# 6. rn_id - список количества операций для каждого клиента
# 6. array_data - данные преобразоанные к numpy-массиву
# 6. split_array - сгрупированные по клиентам данные преобразованные

# обьявлем функцию
def torow_func(dict_params):
    pd_data = dict_params['data']
    n_last = dict_params['n_last']

    # извлекаем список "id" клиентов
    list_id = pd_data['id'].unique().tolist()
    
    # извлекаем список признаков из данных        
    list_features = pd_data.columns.drop(['id','rn'])

    # формируем словарь для зашифрованных признаков
    dict_features = {}
    
    # заполним словарь dict_features
    num_f=0
    for feature in list_features:
        # шифруем признак: fk = "feature_agg_function"
        for num_feature in range(1,n_last+1):
            dict_features['f'+str(num_feature+num_f)] = feature+'_'+str(num_feature)
        num_f+=n_last
    

    # формируем словарь rn_id
    rn_id = pd_data.groupby('id')['id'].count().to_list()

    # для улучшения производительности преобразуем DataFrame в array-массив
    array_data = np.array(pd_data.iloc[:,2:]).transpose()

    # "порежем массив" по длине кредитной истории клиента
    split_array = np.array_split(array_data, np.cumsum(rn_id),axis=1)
    
    # определим порядок последующих преобразований в функции
    def transform_array(array_id):
        # обратим порядок клиентских операций 
        reverse_array_id = array_id[::,::-1]
        # выбрем после n операций клиента
        list_n_last = reverse_array_id[::,:n_last]
        # если клиенских операций было меньше чем n_last
        # дополним недастающие нулями и преобразуем данные к строке
        if len(list_n_last[0])<n_last:
            full_list_n_last = np.hstack((list_n_last,np.zeros((list_n_last.shape[0],n_last-len(list_n_last[0])),dtype='int64')))
            # преобразуем список к строке
            full_list_n_last = full_list_n_last.reshape(-1)
        else:
            full_list_n_last = list_n_last.reshape(-1)
        return full_list_n_last

    # применим transform_array преобразование к списку split_array
    list_data = np.array(list(map(transform_array,split_array)))[:-1]
    
    # преобразуем полученные данные к dataframe
    dataframe = pd.DataFrame(data=list_data, columns=dict_features.keys())

    # добавим столбец id
    dataframe.insert(0,'id',list_id)
    
    return dataframe, dict_features,rn_id


# преобразуем функции в инструмент для преобразования данных (Transformer)
torow_transformer = FunctionTransformer(torow_func)

In [11]:
# функция features_from_transform_data_torow

# Назначение: Извлечение из данных, над которомы совершено 
#             row_fich_transformer() преобразование, признаков  
#             соотвествующих заданному числу последних  
#             опреаций клиента n_last

# Внешние переменные функции: DataFrame
#   Признаки DataFrame:
#       1. n_last - необходимое число последних операций клиента
#       2. n_groups - число групп признаков в transform_data_torow
#       3. N_last - число последних операций клиента показанных в transform_data_torow

# Результат работы функции: 
# 1. list_n_last_features - список признаков в transform_data_torow
#    соотвествующий заданному числу n_last. 
    

# обьявлем функцию
def features_from_transform_data_torow(n_last,n_groups,N_last):
    # создадим список под необходимые признаки
    list_n_last_features = []
    
    # обьявим начальное значение в группе признаков
    n_start = 0
    
    for i in range(n_groups):
        for n in range(n_last):
            list_n_last_features.append(n+n_start)
        n_start+=N_last
    
    return list_n_last_features   

### <span style="color:RoyalBlue">функция diff_feature</span>

In [12]:
# функция diff_feature

# Назначение: Определение дифференциальных характеристик ряда 

# Внешние переменные функции: 
#           1.Series/np.array/list


# Результат работы функции: 
# 1. diff_list - Список из значений:
#                   1.1. speed - скорость изменения ряда;
#                   1.2. accel - ускорение изменения ряда;
#                   1.3. bias - смещение ряда;
#                   1.4. pulse - импульс ряда;

# обьявлем функцию
def diff_feature(data):
    # преобразуем данные к numpy массиву
    data = np.array(data)
    # расчитаем необходимые характеристики
    speed = round(float(np.diff(data,1).mean()),2)
    accel = round(float(np.diff(data,2).mean()),2)
    bias = round(float(np.diff(data,1).sum()),2)
    pulse = round(float(np.diff(data,2).sum()),2)
    # сформируем из найденных значений в список
    diff_list = [speed,accel,bias,pulse]
    return diff_list

### <span style="color:RoyalBlue">функция statistic_features</span>

In [13]:
# функция statistic_features

# Назначение: Извлечение основных статистических характеристик
#             из признаков в исходном DataFrame.

# Внешние переменные функции: DataFrame
#   Признаки DataFrame:
#       1. id
#       2. feature1
#       3. feature2
#       4. feature3
#       ...

# Результат работы функции: 
# 1. dataframe - таблица с данными. 
#    Признаки dataframe:
#       1. id
#       2. feature1_mean
#       3. fearture1_hmean
#       4. feature1_std
#       5. feature1_min
#       6. feature1_25%
#       7. feature1_50%
#       8. feature1_75%
#       9. feature1_max
#       10. feature1_mode
#       11. feature1_frequency_mode
#       12. feature2_mean
#       ...
    
# 2. dict_features - словарь (карта) признаков,
#    в котором отображается сокращения признаков и их расщифровка

# Алгоритм работы функции:
# 1. извлекаем признаки из данных
# 2. формируем карту признаков:
#    2.1. каждый признак кодируется следующим образом: 'fn' где n - порядковый номер признака
#    2.2. полные имена признаков задаются следующим образом:
#         2.2.1 если в исходном dataframe признак бинарный, то: "Исходное имя признака"+"binary"
#         2.2.2 если в исходном dataframe признак не бинарный, то: "Исходное имя признака"+"Статистическая характеристика"
# 3. для каждого клиента по каждому признаку из исходного dataframe расчитываем статистические характеристики
# 4. записываем полученные значение в новый dataframe

# Описание локальных переменных функции:
# 1. dict_agg_function - словарь из агригирующих функций
#       keys: имена для обращения к функциям:
#       values: lamda-функция, соотвествующей статистической характристики
# 2. list_features - список для хранения признаков исхдного dataframe внутри функции
# 3. list_id - список для хранения id клиентов внутри функции
# 4. dict_features - локальная карта признаков
# 5. k - номер признака в dict_features на текущей итерации
# 6. dataframe - результирующий dataframe

# обьявлем функцию
def statistic_features(pd_data):
    # формируем список из функций для статистических преобразований
    # предусмотрим работу функций на случай, если в массиве данных всего 1 строка
    dict_agg_function = {
    'ptp' : lambda x: 0 if len(x) <= 3 else np.ptp(x),
    'mean': lambda x: 0 if len(x) <= 3 else x.mean(), 
    'gmean' : lambda x: stats.gmean(x),   
    'hmean': lambda x: stats.gmean(x),
    # 'pmean25': lambda x: stats.pmean(x,25),
    'pmean50': lambda x: stats.pmean(x,50),
    # 'pmean75': lambda x: stats.pmean(x,75),
    # 'expectile25': lambda x: stats.expectile(x,0.25),
    'expectile50': lambda x: stats.expectile(x),
    # 'expectile75': lambda x: stats.expectile(x,0.75),
    'moment': lambda x: stats.moment(x),
    'std': lambda x: 0 if len(x) <= 3 else np.std(x),
    'min': lambda x: min(x),
    '20%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=20),
    # '30%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=30),
    # '40%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=40),
    '50%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=50),
    # '60%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=60),
    '70%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=70),
    'max': lambda x: max(x),
    'mode': lambda x: statistics.mean(statistics.multimode(x)),
    'frequency_mode': lambda x: round(list(x).count(statistics.multimode(x)[0])*len(statistics.multimode(x))/len(x),2),
    'cov' : lambda x: 0 if len(x) <= 3 else np.cov(x),
    'histogram' : lambda x: 0 if len(x) <= 3 else np.histogram(x)[1].mean(), 
    # 'speed': lambda x: 0 if len(x) <= 3 else diff_feature(x)[0],
    # 'accel': lambda x: 0 if len(x) <= 3 else diff_feature(x)[1],
    # 'bias': lambda x: 0 if len(x) <= 3 else diff_feature(x)[2],
    # 'pulse': lambda x: 0 if len(x) <= 3  else diff_feature(x)[3]
    } 

    # напишем функцию для преобразования массива до статистических характеристик
    def stat_func(array_data): 
        # сформируем лист под результаты преобразования
        list_for_result = []
        # запишем все статистические харкетристики из словаря dict_agg_function
        for func in dict_agg_function.values():
            list_for_result.append(func(array_data))
        return np.array(list_for_result).round(3)

    # напишем функцию для применения функции stat_func к списку
    def submap(list_data):
        # расчитаем количество операция клиента
        max_rn = len(list_data[0])
        # получим статистические характристики массива
        list_stat_features = np.array(list(map(stat_func,list_data))).reshape(-1)
        return np.hstack((max_rn,list_stat_features))
    
    # извлекаем список "id" клиентов
    list_id = pd_data['id'].unique().tolist()

    # извлекаем список признаков из данных        
    list_features = pd_data.columns.drop(['id','rn'])
    
    # формируем словарь для зашифрованных признаков
    dict_features = {'f1':'count'}
    k=1 # порядковый номер защифрованного признака

    # заполним словарь dict_features
    for feature in list_features:
        # шифруем признак: fk = "feature_agg_function"
        for key_function in dict_agg_function.keys():
            k+=1
            dict_features['f'+str(k)] = feature+'_'+key_function

    # формируем список rn_id
    rn_id = pd_data.groupby('id')['id'].count().to_list()

    # для улучшения производительности преобразуем DataFrame в array-массив
    array_data = np.array(pd_data.iloc[:,2:]).transpose()

    # "порежем массив" по длине кредитной истории клиента
    split_array = np.array_split(array_data, np.cumsum(rn_id),axis=1)[:-1]

    # получем статические характеристики признаков
    stat_features = np.array(list(map(submap,split_array)))
    
    # Сформируем dataframe из полученных данных
    dataframe = pd.DataFrame(data=stat_features, columns=dict_features.keys())

    # добавим столбец id
    dataframe.insert(0,'id',list_id)

    return dataframe, dict_features

# преобразуем функции в инструмент для преобразования данных (Transformer)
stat_transformer = FunctionTransformer(statistic_features)

### <span style="color:RoyalBlue">функция corr_transform_to_force</span>

In [14]:
# функция corr_transform_to_force

# Назначение: из матрицы взаимных корреляций
#             выделить не корелирующие признаки

# Внешние переменные функции: 
#           1. df.corr() - матрица корреляций
#           2. threshold - порог значимости корреляции:
#               значение коэффициента корреляции, больше которого
#               признаки считаются скоррелированными.

# Пояснение: 
# Под силой корреляции будем понимать следующее: если коэффициент 
# коррелиции между признаками больше значения threshold, то принимаем,
# что между признаками сильная корреляционная связь значение коэффициента 
# коррелияции заменяем на 1, иначе корреляционная связь слабая и значение 
# коээфициента корреляции заменяем на 0

# Результат работы функции: 
# 1. corr_matrix - матрица корреляций(отражает силу корреляции)
# 2. list_ncorr_features - список не скореллированных признаков
# 3. corr_force - сила корреляции всей матрицы: отношение числа скоррелированных 
# признаков к числу всех признаков в матрице

# Описание локальных переменных функции:
# 1. coor_force - функция преобазующая значение
#        коэффициента корряляции в силу корреляции
# 2. corr_matrix - матрица отражающая силу корряляции между признаками
# 3. max_corr - максимальное число взаимных корреляций между признаками
# 4. list_ncorr_features - список не коррелируемых признаков


# обьявлем функцию
def corr_transform_to_force(matrix,threshold=0.7):
    list_features = matrix.index.tolist()
    
    
    # создадим функцию для разметки матрицы корреляции
    # 1 - корреляция признаков выше порога значимости threshold
    # 0 - корреляция признаков ниже порога значимости threshold
    corr_force = lambda x: 1 if x >threshold else 0
    # выполним разметку матрицы корреляции
    corr_matrix = matrix.map(lambda x: corr_force(x))
    
    # алгоритм отбора не коррелиарных признаков:
    #   1. Найдем признак с наибольшим числом взаимных корреляций
    #   2. удалим найденный признак
    #   3. составим матрицу корреляций из отсавшися признаков
    #   4. повторяем пункты 1-3 до тех пор пока в матрице не останутся 
    #       не коррелированные признаки

    # ищем наибольшее число взаимных корреляций среди признаков
    max_corr = corr_matrix.sum().max()

    while max_corr > 1:
        # определяем признак с наибольшим числом взаимных корреляций
        max_corr_feature = corr_matrix.sum()[corr_matrix.sum()==corr_matrix.sum().max()].index[0]
        # удалем признак из матрицы корреляций
        corr_matrix = corr_matrix.drop(max_corr_feature).drop(max_corr_feature,axis=1)
        max_corr = corr_matrix.sum().max()
    # запишем не скоррелированные признаки в список
    list_ncorr_features = corr_matrix.index.tolist()
    # найдем силу корреляции всей матрицы как отношение
    # количества скоррелированных признаков к всмеу количеству признаков
    corr_force = round(1-len(list_ncorr_features)/len(list_features),3)
    return corr_matrix, list_ncorr_features, corr_force

### <span style="color:RoyalBlue">функция search_DBSCAN_parameters</span>

In [15]:
# функция search_DBSCAN_parameters

# Назначение: Для подбора eps и min_samples параметров,
#               функция "прогоняет" DBSCAN кластеризацию 
#               с параметрами eps и min_samples
#               примающими значения из заданного диапазона.

# Внешние переменные функции: 
#           1. data - dataframe для кластеризации
#           2. r1 - начало диапазона
#           3. r2 - конец диапазона  
#           4. n - предпалагамое число кластеров      

# Результат работы функции: 
# 1. data_cluster - кластеризация данных при различных 
#       значениях параметров eps и min_samples

# Описание локальных переменных функции:
# 1. parametr_range - диапазон изменения параметров
# 2. dataframe_columns - колонки в результирующем dataframe
# 3. data_cluster - результрующий dataframe
# 4. index_cluster - текущая позиция в data_cluster
# 5. clustering - кластеризатор
# 6. list_cluster_values - список для заполнения текущими 
#                          значениями data_cluster

# обьявлем функцию
def search_DBSCAN_parameters(dataframe,r1,r2,n=3):
    # задаем диапозон измениния параметров
    parameter_range = range(r1,r2)
    # формируем заготовку для результирующего dataframe
    dataframe_columns = ['eps','min_samples',-1,0,1]
    # проверим что задано не меньше минимального количества кластеров
    if n<=3: 
        data_cluster = pd.DataFrame(columns=dataframe_columns)
    else: 
        for claster in range(4,n+1):
            dataframe_columns.append(claster-2)
        data_cluster = pd.DataFrame(columns=dataframe_columns)
    # задаем начально значение индекса в data_cluster
    index_cluster = 0

    # для подсчета обьектов в кластерах создадим dataframe
    dataframe_count = pd.DataFrame()
    
    # "прогоняем" DBSCAN кластеризациию по диапазону параметров
    for eps in parameter_range:
        
        for min_samples in parameter_range:
            print('current eps:',eps,'  current min_samples:', min_samples, end='\r')
            # запускаем кластеризацию с текущими параметрами
            clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(dataframe)
            # добавлем к данным столбец с разметкой
            dataframe_count['clater'] = clustering.labels_
            # формируем пустой список для заполнения
            list_cluster_values = []
            # добавлеям в список текущие параметры
            list_cluster_values.append(eps)
            list_cluster_values.append(min_samples)
            # добавлем в список количество обьктов в каждом кластере
            for column in dataframe_columns[2:]:
                list_cluster_values.append(len(dataframe_count['clater'][dataframe_count['clater']==column]))
                
            # заполняем dataframe  текущими данными
            data_cluster.loc[index_cluster] = list_cluster_values
            index_cluster +=1
            # сбрасываем dataframe_count
            dataframe_count = pd.DataFrame()
    return data_cluster

### <span style="color:RoyalBlue">функция generate_samples</span>

In [16]:
# функция generate_samples

# Назначение: для генерации индексов выборок данных

# Внешние переменные функции: 
#           1. max - определяет максимальное значение множества
#               из которого формируются выборки
#           2. n - количество выборок
#           3. k - мощность одной выборки
     

# Результат работы функции: 
# 1. samples_list - список с выбороками

# алгоритм работы:
# 1. задаем отрезок натурального ряда N мощностью max и добавлем в него 0.
#       In = N U {0}, I = {0,1,2,3,4,5,..,max}
# 2. если мощность множества In больше, необходимого количества элементов
#       cardo(In) > n x k , то из множества In формируем n случайных выборок
# размера k без повторения.
# 3. если мощность множества In меньше, необходимого количества элементов
#       cardo(In) < n x k , то из множества In формируем случайные выборки
# размера k без повторения, до тех пор пока не закончится множество In.
# После, добираем недостающее количество выборок случайными выборками 
# размера k из множества In с повторением (bootstrap метод).


# обьявлем функцию
def generate_samples(max,n,k,random_state = None):
    # создадим список под результат
    samples_list = []
    # формируем множество натуральных числе от 0 до max
    In = list(range(max+1))
    # Будем выполнять код пока не наберем необходимого количества выборок
    # нарушим порядок в множестве
    In = shuffle(In,random_state=random_state)
    # random.shuffle(In)

    # задаим границы извлечения данных из In
    In_start = 0
    In_end = k
    while len(samples_list) < n:
        # сформируем список под одну выборку
        sample = []
        # первые списки будем наполнять значениеми из множества In
        # без повторения, до тех пор пока все значения из множества In
        # не распределяться по выборкам
        if len(In)-In_end >= 0:
            sample.extend(In[In_start:In_end])
        else:                    
            # если элементов во множестве In недостаточно,
            # запоняем выборку "остатками" 
            sample.extend(In[In_start:])

            # остальные данные заполняем методом bootstrap
            # выполнем код пока не заполним выборку k значениями
            while len(sample) < k:
                # генерируем случайное число из диапазона от 0 до len(In)-1
                random_index = randint(0,len(In)-1)
                # добавляем значение из множества In с индексом random_index
                # в список index_list
                sample.append(In[random_index])

        # после того как мы набрали значения в выборку отправлем ее в samples_list
        samples_list.append(sample)
        # переходим к следующим данным в множестве In
        In_start+=k
        In_end+=k

    return samples_list

### <span style="color:RoyalBlue">функция my_train_test_split</span>

In [17]:
def my_train_test_split(X,y,random_state=42,train_size=0.8,):
    # если разбиение без стратификации

    # зададим число элементов в выборке train
    len_train = round(len(y)*train_size)
    # формируем множество натуральных чисел от 0 до max
    list_random_index = list(range(len(y)))
    # нарушим порядок в множестве
    list_random_index = shuffle(list_random_index,random_state=random_state)
    # формируем список индексов под train выборку
    train_samples = list_random_index[:len_train]
    # формируем список индексов под test выборку
    test_samples = list_random_index[len_train:]
    # выполнем код пока не заполним выборку k значениями
   
    X_train = X.iloc[train_samples]
    y_train = y.iloc[train_samples]
    X_test = X.iloc[test_samples]
    y_test = y.iloc[test_samples]

    return X_train, y_train, X_test, y_test

### <span style="color:RoyalBlue">функция class_1_percent_samples</span>

In [18]:
# функция class_1_percent_samples

# Назначение: для генерации индексов сбалансированных выборок

# Внешние переменные функции: 
#           1. data_target - массив из id и значений класса
#           2. class_1_percent - процент класса 1 в результирующей выборке
#           3. random_state - параметр для обеспечения воспроизваодимости функции
     

# Результат работы функции: 
# 1. samples_list - список со сблансированными выбороками

# обьявлем функцию
def class_1_percent_samples(data_target,class_1_percent,random_state = None):
    # приведем данные к нужной форме
    data_target = pd.DataFrame(data=np.array(data_target),columns =['id','flag'])
    
    # разделим клиентов  по признаку flag
    flag_0 = data_target[data_target['flag']==0].reset_index(drop=True)
    flag_1 = data_target[data_target['flag']==1].reset_index(drop=True)

    # определим класс большинства
    if flag_1.shape[0] > flag_0.shape[0]:
        majority_class = flag_1
        minority_class = flag_0
        # расчитаем необходимую величину выборки majority_class
        majority_class_size = round(minority_class.shape[0]*(class_1_percent)/(1-class_1_percent))
        # с помощью функции generate_samples сформируем выборку для majority_class
        samples_majority_class= generate_samples(majority_class.shape[0]-1,1,majority_class_size,random_state=random_state)
    else:
        majority_class = flag_0
        minority_class = flag_1
        # расчитаем необходимую величину выборки majority_class
        majority_class_size = round(minority_class.shape[0]*(1-class_1_percent)/(class_1_percent))
        # с помощью функции generate_samples сформируем выборку для majority_class
        samples_majority_class= generate_samples(majority_class.shape[0]-1,1,majority_class_size,random_state=random_state)
    
    # сформируем список выбороки с заданным процентом класс 1
    samples_list_id = minority_class['id'].values.tolist()+majority_class['id'].iloc[samples_majority_class[0]].tolist()

    return samples_list_id

## <span style="color:DodgerBlue">Преобразование признаков и анализ признаков</span>

### <span style="color:RoyalBlue">Разделим признаки на 6 под пространств</span>

**date features:**  
1. pre_since_opened	-	Дней с даты открытия кредита до даты сбора данных (бинаризовано**)	  
2. pre_since_confirmed	-	Дней с даты подтверждения информации по кредиту до даты сбора данных (бинаризовано**)	  
3. pre_pterm	-	Плановое количество дней с даты открытия кредита до даты закрытия (бинаризовано**)	  
4. pre_fterm	-	Фактическое количество дней с даты открытия кредита до даты закрытия (бинаризовано**)	  
5. pre_till_pclose	-	Плановое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано**)	  
6. pre_till_fclose	-	Фактическое количество дней с даты сбора данных до даты закрытия кредита (бинаризовано**)
7. pclose_flag	-	Флаг: плановое количество дней с даты открытия кредита до даты закрытия не определено 	  
8. fclose_flag	-	Флаг: фактическое количество дней с даты открытия кредита до даты закрытия не определено

**late payments features:**  
1. pre_loans5	-	Число просрочек до 5 дней (бинаризовано**)	  
2. pre_loans530	-	Число просрочек от 5 до 30 дней (бинаризовано**)	  
3. pre_loans3060	-	Число просрочек от 30 до 60 дней (бинаризовано**)	  
4. pre_loans6090	-	Число просрочек от 60 до 90 дней (бинаризовано**)	  
5. pre_loans90	-	Число просрочек более, чем на 90 дней (бинаризовано**)	  
6. is_zero_loans5	-	Флаг: нет просрочек до 5 дней	  
7. is_zero_loans530	-	Флаг: нет просрочек от 5 до 30 дней	  
8. is_zero_loans3060	-	Флаг: нет просрочек от 30 до 60 дней	  
9. is_zero_loans6090	-	Флаг: нет просрочек от 60 до 90 дней	  
10. is_zero_loans90	-	Флаг: нет просрочек более, чем на 90 дней	  
11. pre_loans_total_overdue	-	Текущая просроченная задолженность (бинаризовано**)	  
12. pre_loans_max_overdue_sum	-	Максимальная просроченная задолженность (бинаризовано**)	


**credit features:** 
1. pre_loans_credit_limit	-	Кредитный лимит (бинаризовано**)	  
2. pre_loans_next_pay_summ	-	Сумма следующего платежа по кредиту (бинаризовано**)	  
3. pre_loans_outstanding	-	Оставшаяся невыплаченная сумма кредита (бинаризовано**)	  
4. pre_loans_credit_cost_rate	-	Полная стоимость кредита (бинаризовано**)	  
  

**relative features:**  
1. pre_util	-	Отношение оставшейся невыплаченной суммы кредита к кредитному лимиту (бинаризовано**)	  
2. pre_over2limit	-	Отношение текущей просроченной задолженности к кредитному лимиту (бинаризовано**)	  
3. pre_maxover2limit	-	Отношенение максимальной просроченной задолженности к кредитному лимиту (бинаризовано**)	  
4. is_zero_util	-	Флаг: отношение оставшейся невыплаченной суммы кредита к кредитному лимиту равняется 0	  
5. is_zero_over2limit	-	Флаг: отношение текущей просроченной задолженности к кредитному лимиту равняется 0	  
6. is_zero_maxover2limit	-	Флаг: отношение максимальной просроченной задолженности к кредитному лимиту равняется 0	  

**payments features:**   
1. enc_paym_{0..N}	-	Статусы ежемесячных платежей за последние N месяцев (закодировано***)	  

**service features:**  
1. enc_loans_account_holder_type	-	Тип отношения к кредиту (закодировано***)	  
2. enc_loans_credit_status	-	Статус кредита (закодировано***)	  
3. enc_loans_account_cur	-	Валюта кредита (закодировано***)	  
4. enc_loans_credit_type	-	Тип кредита (закодировано***)	  

  ** область значений поля разбивается на N непересекающихся промежутков, каждому промежутку случайным образом ставится в соответствие уникальный номер от 0 до N-1, значение поля заменяется номером промежутка, которому оно принадлежит  
  *** каждому уникальному значению поля случайным образом ставится в соответствие уникальный номер от 0 до K, значение поля заменяется номером этого значения		

In [19]:
# сформируем списки признаков каждого подпространства
date_features = ['id','rn','pre_since_opened','pre_since_confirmed','pre_pterm','pre_fterm',
                 'pre_till_pclose','pre_till_fclose','pclose_flag','fclose_flag']
late_features = ['id','rn','pre_loans5','pre_loans530','pre_loans3060','pre_loans6090',
                 'pre_loans90','is_zero_loans5','is_zero_loans530','is_zero_loans3060',
                 'is_zero_loans6090','is_zero_loans90','pre_loans_total_overdue','pre_loans_max_overdue_sum']
credit_features = ['id','rn','pre_loans_credit_limit','pre_loans_next_pay_summ','pre_loans_outstanding','pre_loans_credit_cost_rate']

relative_features = ['id','rn','pre_util','pre_over2limit','pre_maxover2limit','is_zero_util',
                 'is_zero_over2limit','is_zero_maxover2limit']

payments_features = ['id','rn'] + train_data.columns[30:55].to_list()
service_features = ['id','rn'] + train_data.columns[55:59].to_list()

### <span style="color:RoyalBlue">Выполним torow-transform преобразование</span>

In [ ]:
# составим словарь небходимых данных и их признаков
dict_torow_data = {
    'date_torow' : date_features,
    'late_torow' : late_features,
    'credit_torow' : credit_features,
    'relative_torow' : relative_features,
    'payments_torow': payments_features,
    'service_torow': service_features}
# сформируем данные за последние 25 операций клиентов
for space_features in dict_torow_data.keys():
    clear_output()
    # добавим индекацию процесса
    print('Current space features:',space_features)
    # сформируем данные для преобразования
    data_to_transform = train_data[dict_torow_data[space_features]]
    print('Start transform')
    data_torow = torow_transformer.transform({'data':data_to_transform,'n_last':25})[0]
    # сохраним преобразованные данные на диск для быстрого воспроизведения
    print('Start save')
    fp.write('base_models/torow/'+space_features,data_torow)
    # удалим использованные данные
    del data_to_transform
    gc.collect()

Current space features: service_torow
Start transform
Start save


### <span style="color:RoyalBlue">Выполним stat-transform преобразование</span>

In [ ]:
# составим словарь небходимых данных и их признаков
dict_stat_data = {
    'date_stat' : date_features,
    'late_stat' : late_features,
    'credit_stat' : credit_features,
    'relative_stat' : relative_features,
    'payments_stat': payments_features,
    'service_stat': service_features}
# сформируем данные за последние 25 операций клиентов
for space_features in dict_stat_data.keys():
    clear_output()
    # добавим индекацию процесса
    print('Current space features:',space_features)
    # сформируем данные для преобразования
    data_to_transform = train_data[dict_stat_data[space_features]]
    print('Start transform')
    data_stat = stat_transformer.transform(data_to_transform)[0]
    # сохраним преобразованные данные на диск для быстрого воспроизведения
    print('Start save')
    fp.write('base_models/stat/'+space_features,data_stat)
    # удалим использованные данные
    del data_to_transform, data_stat
    gc.collect()

Current space features: service_stat
Start transform
Start save


### <span style="color:RoyalBlue">Корреляция признаков</span>

In [21]:
# составим словарь небходимых данных и их признаков
list_space_features = ['date','late','credit','relative','payments','service']

# сформируем данные за последние 25 операций клиентов
for space_features in list_space_features:
    clear_output()
    # добавим индекацию процесса
    print('Current space features:',space_features)
    # подгружаем данные
    drop_id_stat_data = fp.ParquetFile('base_models/data/'+space_features+'_stat').to_pandas().drop('id',axis=1)
    # сформируем матрицу взаимных корреляций
    print('Found corr matrix')
    corr_matrix = drop_id_stat_data.corr()
    # сохраним матрицу взаимных корреляций на диск для быстрого воспроизведения
    print('Start save matrix')
    fp.write('base_models/data/corr_matrix_'+space_features,corr_matrix)
    # удалим использованные данные
    del drop_id_stat_data, corr_matrix
    gc.collect()

Current space features: service
Found corr matrix
Start save matrix


## <span style="color:DodgerBlue">Формирование данных для обучения модели</span>

### <span style="color:RoyalBlue">Формирование тренировочного, валидационного, тестового и отложенного набора данных</span>

In [26]:
# для формирования наборов данных выберем dataset service_stat
transform_data_stat = fp.ParquetFile('features/base_models/service_stat').to_pandas()

In [27]:
# выделим отложенный набор данных для проверки результирующей модели
X_atrain, y_atrain, X_hold, y_hold = my_train_test_split(transform_data_stat.drop('id',axis=1),target['flag'], train_size=0.95)

In [28]:
# Оставшиеся данные раздедлим на тренировочный и тестовый наборы
X_train, y_train, X_test, y_test = my_train_test_split(X_atrain, y_atrain, train_size=0.25)

In [29]:
# Для валидации модели выделим данные из тренировочного набора
X_train, y_train, X_valid, y_valid = my_train_test_split(X_train,y_train, train_size=0.9)

In [30]:
# проверим сбалансированность данных по признаку flag
print('target', target['flag'].value_counts(normalize=True))
print('train', y_train.value_counts(normalize=True))
print('valid', y_valid.value_counts(normalize=True))
print('test',y_test.value_counts(normalize=True))
print('hold',y_hold.value_counts(normalize=True))

target flag
0    0.964519
1    0.035481
Name: proportion, dtype: float64
train flag
0    0.964242
1    0.035758
Name: proportion, dtype: float64
valid flag
0    0.96487
1    0.03513
Name: proportion, dtype: float64
test flag
0    0.964631
1    0.035369
Name: proportion, dtype: float64
hold flag
0    0.963947
1    0.036053
Name: proportion, dtype: float64


In [31]:
# проверим размерности полученных наборов
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)
print(X_hold.shape,y_hold.shape)

(641250, 69) (641250,)
(71250, 69) (71250,)
(2137500, 69) (2137500,)
(150000, 69) (150000,)


In [32]:
X_train.shape[0]+X_valid.shape[0]

712500

Для воспроизводимости кода и корректного сравнения различных моделей  
завиксируем id для тренировочного, валидационного и тестового наборов данных.

In [33]:
# Спрячем данные в DataDfame для последующего сохраниения в csv
train_id = pd.DataFrame(y_train.index)
valid_id = pd.DataFrame(y_valid.index)
test_id = pd.DataFrame(y_test.index)
hold_id = pd.DataFrame(y_hold.index)

In [37]:
# Сохраняем полученные DataFrame
train_id.to_csv('features/base_models/reproduce/train_id.csv',index=False)
valid_id.to_csv('features/base_models/reproduce/valid_id.csv',index=False)
test_id.to_csv('features/base_models/reproduce/test_id.csv',index=False)
hold_id.to_csv('features/base_models/reproduce/hold_id.csv',index=False)

In [38]:
# подгрузим DataFrame с id для тренировочного/валидационного/тестового набора данных
train_id = pd.read_csv('features/base_models/reproduce/train_id.csv')['0'].to_list()
valid_id = pd.read_csv('features/base_models/reproduce/valid_id.csv')['0'].to_list()
test_id = pd.read_csv('features/base_models/reproduce/test_id.csv')['0'].to_list()
hold_id = pd.read_csv('features/base_models/reproduce/hold_id.csv')['0'].to_list()

In [39]:
# подгрузим данные целевой переменной
target = pd.read_csv('data_source/train_data/train_target.csv')

In [40]:
# сохраним тренировочный валидационный и тестовый целевой переменной
# чтобы иметь возможность к ними обратиться
target.loc[train_id].to_csv('target/target_train.csv',index=False)
target.loc[valid_id].to_csv('target/target_valid.csv',index=False)
target.loc[test_id].to_csv('target/target_test.csv',index=False)
target.loc[hold_id].to_csv('target/target_hold.csv',index=False)

### <span style="color:RoyalBlue">Формирование тренировочного, валидационного, тестового и отложенного набора на transform_data_torow_25 данных</span>

In [42]:
# составим список пространств признаков
list_spaces = ['date_torow','late_torow','credit_torow','relative_torow','payments_torow','service_torow']
# составим словарь небходимых наборов данных
dict_datasets = {
    'train' : pd.read_csv('features/base_models/reproduce/train_id.csv')['0'].to_list(),
    'valid' : pd.read_csv('features/base_models/reproduce/valid_id.csv')['0'].to_list(),
    'test' : pd.read_csv('features/base_models/reproduce/test_id.csv')['0'].to_list(),
    'hold' : pd.read_csv('features/base_models/reproduce/hold_id.csv')['0'].to_list()
}
# разобьем torow данные на обучающий/валидационный/тестовый/отложенный наборы
for space_features in list_spaces:
    for name_dataset in dict_datasets.keys():
        clear_output()
        # добавим индекацию процесса
        print('Current space features:',space_features)
        print('Current name_dataset:',name_dataset)
        # загрузим необходимые данные
        dataset_id = dict_datasets[name_dataset]
        dataset = fp.ParquetFile('features/base_models/torow/'+space_features).to_pandas().set_index('id').iloc[dataset_id]
        # сохраним отборанные данные
        print('Start save '+space_features+'_'+name_dataset)
        fp.write('features/base_models/torow/'+space_features+'_'+name_dataset,dataset)
        # удалим использованные данные
        del dataset, dataset_id
        gc.collect()

Current space features: service_torow
Current name_dataset: hold
Start save service_torow_hold


### <span style="color:RoyalBlue">Формирование тренировочного, валидационного, тестового и отложенного набора на transform_data_stat данных</span>

In [43]:
# составим список простраств признаков
list_spaces = ['date_stat','late_stat','credit_stat','relative_stat','payments_stat','service_stat']
# составим словарь небходимых наборов данных
dict_datasets = {
    'train' : pd.read_csv('features/base_models/reproduce/train_id.csv')['0'].to_list(),
    'valid' : pd.read_csv('features/base_models/reproduce/valid_id.csv')['0'].to_list(),
    'test' : pd.read_csv('features/base_models/reproduce/test_id.csv')['0'].to_list(),
    'hold' : pd.read_csv('features/base_models/reproduce/hold_id.csv')['0'].to_list()
}
# разобьем stat данные на обучающий/валидационный/тестовый/отложенный наборы
for space_features in list_spaces:
    for name_dataset in dict_datasets.keys():
        clear_output()
        # добавим индекацию процесса
        print('Current space features:',space_features)
        print('Current name_dataset:',name_dataset)
        # загрузим необходимые данные
        dataset_id = dict_datasets[name_dataset]
        dataset = fp.ParquetFile('features/base_models/stat/'+space_features).to_pandas().set_index('id').iloc[dataset_id]
        # сохраним отборанные данные
        print('Start save '+space_features+'_'+name_dataset)
        fp.write('features/base_models/stat/'+space_features+'_'+name_dataset,dataset)
        # удалим использованные данные
        del dataset, dataset_id
        gc.collect()

Current space features: service_stat
Current name_dataset: hold
Start save service_stat_hold
